In [1]:
import sys
sys.path.append('../src/general/')
import CreateDataName as cn
import AssessModelModified as am
import plotting as rfplt
import ReadRoutinesModified as rr

import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

import gc

import os


In [2]:
plt.rcParams.update({'font.size': 10})
plt.rc('font', family='sans serif')
plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')

In [3]:
run_vars = {'dimension':3, 'lat':True , 'lon':True , 'dep':True , 'current':True , 'bolus_vel':True , 'sal':True , 'eta':True , 'density':True , 'poly_degree':1, 'StepSize':1, 'predict':'DelT'}
data_prefix = ''
model_prefix = 'alpha.001_'

TrainModel = True  

DIR = '../../data/raw/'    
MITGCM_filename=DIR+'cat_tave.nc'
density_file = DIR+'DensityData.npy'
clim_filename = DIR+'ncra_cat_tave.nc'

print('run_vars; '+str(run_vars))
print('model_prefix ; '+str(model_prefix))

run_vars; {'dimension': 3, 'lat': True, 'lon': True, 'dep': True, 'current': True, 'bolus_vel': True, 'sal': True, 'eta': True, 'density': True, 'poly_degree': 1, 'StepSize': 1, 'predict': 'DelT'}
model_prefix ; alpha.001_


In [4]:
data_name = data_prefix + cn.create_dataname(run_vars)
model_name = model_prefix+data_name
print(f"Model name: {model_name}")

Model name: alpha.001_3dLatLonDepUVBolSalEtaDnsPolyDeg1_Step1_PredictDelT


In [5]:
print('reading in data')
norm_inputs_tr, norm_inputs_val, norm_inputs_te, norm_outputs_DelT_tr, norm_outputs_DelT_val, norm_outputs_DelT_te, \
norm_outputs_Temp_tr, norm_outputs_Temp_val, norm_outputs_Temp_te, orig_temp_tr, orig_temp_val, clim_temp_tr, clim_temp_val \
          = rr.ReadMITGCM(MITGCM_filename, clim_filename, density_file, 0.7, 0.9, data_name, run_vars, plot_histograms=False)
del norm_inputs_te
del norm_outputs_DelT_te
del norm_outputs_Temp_te

reading in data
(18000, 42, 78, 11)
0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
*********************************
Number of training & validation samples > 0.0005: [171979], [52235]
Number of training & validation samples > 0.001:  [111033], [34604]
Number of training & validation samples > 0.002:  [82846], [25565]
Number of training & validation samples > 0.0025: [76097], [23495]
Number of training & validation samples > 0.003:  [70810], [22054]
Number of training & validation samples > 0.004:  [62849], [19632]
Number of training & validation samples > 0.005:  [56885], [17853]
*********************************
highest and lowest values in training data:   0.26704645, -0.23875284
highest and lowest values in validation data: 0.23960555, -0.26727915
mean of train and val sets : -5.388849e-05, 0.0001357027
std  of train and val sets : 0.01036672, 0.01

In [6]:
if run_vars['predict'] == 'DelT':
   norm_outputs_tr  = norm_outputs_DelT_tr
   norm_outputs_val = norm_outputs_DelT_val

elif run_vars['predict'] == 'Temp':
   norm_outputs_tr = norm_outputs_Temp_tr
   norm_outputs_val = norm_outputs_Temp_val

print(norm_inputs_tr.shape)
print(norm_inputs_val.shape)
print(norm_outputs_val.shape)

(648440, 228)
(199520, 228)
(199520, 1)


In [7]:
print('setting up model')
pkl_filename = '../outputs/models/'+model_name+'_pickle.pkl'
if TrainModel:
    print('training model')
    
    alpha_s = [0.001]
    parameters = [{'alpha': alpha_s}]
    n_folds=3
   
    lr = linear_model.Ridge(fit_intercept=False)
    
    lr = GridSearchCV(lr, param_grid=parameters, cv=n_folds, scoring='neg_mean_squared_error', refit=True)
    
    # fit the model
    lr.fit(norm_inputs_tr, norm_outputs_tr)
    lr.get_params()

    # Write info on Alpha in file    
    info_filename = '../outputs/models/'+model_name+'_info.txt'
    info_file=open(info_filename,"w")
    info_file.write("Best parameters set found on development set:\n")
    info_file.write('\n')
    info_file.write(str(lr.best_params_)+'\n')
    info_file.write('\n')
    info_file.write("Grid scores on development set:\n")
    info_file.write('\n')
    means = lr.cv_results_['mean_test_score']
    stds = lr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, lr.cv_results_['params']):
        info_file.write("%0.3f (+/-%0.03f) for %r \n" % (mean, std * 2, params))
    info_file.write('')

    # Store coeffs in an npz file
    coef_filename = '../outputs/models/'+model_name+'_coefs.npz'
    np.savez( coef_filename, np.asarray(lr.best_estimator_.intercept_), np.asarray(lr.best_estimator_.coef_) )

    # pickle it
    with open(pkl_filename, 'wb') as pckl_file:
        pickle.dump(lr, pckl_file)

setting up model
training model


In [8]:
with open(pkl_filename, 'rb') as file:
    print('opening '+pkl_filename)
    lr = pickle.load(file)

# predict values
print('predict values')
norm_lr_predicted_tr = lr.predict(norm_inputs_tr).reshape(-1,1).astype('float64')
norm_lr_predicted_val = lr.predict(norm_inputs_val).reshape(-1,1).astype('float64')

opening ../outputs/models/alpha.001_3dLatLonDepUVBolSalEtaDnsPolyDeg1_Step1_PredictDelT_pickle.pkl
predict values


In [9]:
print('predict values')
norm_lr_predicted_tr = lr.predict(norm_inputs_tr).reshape(-1,1).astype('float64')
norm_lr_predicted_val = lr.predict(norm_inputs_val).reshape(-1,1).astype('float64')

predict values


In [10]:
mean_std_file = '../outputs/logs/SinglePoint_'+data_name+'_MeanStd.npz'
zip_mean_std_file = mean_std_file+'.gz' 
if os.path.isfile(mean_std_file):
   mean_std_data = np.load(mean_std_file)
elif os.path.isfile(zip_mean_std_file):
   os.system("gunzip %s" % (zip_mean_std_file))
   mean_std_data = np.load(mean_std_file)
   os.system("gunzip %s" % (mean_std_file))
input_mean  = mean_std_data['arr_0']
input_std   = mean_std_data['arr_1']
output_DelT_mean = mean_std_data['arr_2']
output_DelT_std  = mean_std_data['arr_3']
output_Temp_mean = mean_std_data['arr_4']
output_Temp_std  = mean_std_data['arr_5']

In [11]:
if run_vars['predict'] == 'DelT':
   output_mean = output_DelT_mean
   output_std = output_DelT_std
elif run_vars['predict'] == 'Temp':
   output_mean = output_Temp_mean
   output_std = output_Temp_std

# define denormalising function
def denormalise_data(norm_data,mean,std):
    denorm_data = norm_data * std + mean
    return denorm_data

# denormalise the predictions and true outputs   
denorm_lr_predicted_tr = denormalise_data(norm_lr_predicted_tr, output_mean, output_std)
denorm_lr_predicted_val = denormalise_data(norm_lr_predicted_val, output_mean, output_std)
denorm_outputs_tr = denormalise_data(norm_outputs_tr, output_mean, output_std)
denorm_outputs_val = denormalise_data(norm_outputs_val, output_mean, output_std)

del norm_outputs_tr
del norm_outputs_val
gc.collect()

106

In [12]:
#------------------
# Assess the model
#------------------
# Calculate 'persistance' score - persistence prediction is just zero everywhere as we're predicting the trend
predict_persistance_tr = np.zeros(denorm_outputs_tr.shape)
predict_persistance_val = np.zeros(denorm_outputs_val.shape)

if run_vars['predict'] == 'DelT':
   np.save('./True_DelT', denorm_outputs_tr)
   np.save('./Predicted_DelT', denorm_lr_predicted_tr)
elif run_vars['predict'] == 'Temp':
   np.save('./True_T', denorm_outputs_tr)
   np.save('./Predicted_T', denorm_lr_predicted_tr)

print('get stats')
am.get_stats(model_name, 
             name1='Training', truth1=denorm_outputs_tr, exp1=denorm_lr_predicted_tr, pers1=predict_persistance_tr,
             name2='Validation', truth2=denorm_outputs_val, exp2=denorm_lr_predicted_val, pers2=predict_persistance_val,
             name='TrainVal')

print('plot results')
top    = max(max(denorm_outputs_tr), max(denorm_lr_predicted_tr), max(denorm_outputs_val), max(denorm_lr_predicted_val))
top    = top + 0.1*abs(top)
bottom = min(min(denorm_outputs_tr), min(denorm_lr_predicted_tr), min(denorm_outputs_val), min(denorm_lr_predicted_val))
bottom = bottom - 0.1*abs(top)
am.plot_scatter(model_name, denorm_outputs_tr, denorm_lr_predicted_tr, name='train', top=top, bottom=bottom, text='(a)')
am.plot_scatter(model_name, denorm_outputs_val, denorm_lr_predicted_val, name='val', top=top, bottom=bottom, text='(b)')

##-----------------------------------------------------------------------------------------------------------------
## if training to predict DelT, Calc Stats on temperature rather than DelT for comparison of CCs with other papers
##-----------------------------------------------------------------------------------------------------------------

if run_vars['predict'] == 'DelT':
   # denormalise inputs and true outputs   
   denorm_outputs_Temp_tr = denormalise_data(norm_outputs_Temp_tr, output_Temp_mean, output_Temp_std)
   denorm_outputs_Temp_val = denormalise_data(norm_outputs_Temp_val, output_Temp_mean, output_Temp_std)

   # Take original Temp value as persistence forecast, and add DelT prediction to original value for predicted Temp field 
   predict_persistance_Temp_tr = orig_temp_tr.reshape(-1,1)
   predict_persistance_Temp_val = orig_temp_val.reshape(-1,1)
   denorm_lr_predicted_Temp_tr = orig_temp_tr.reshape(-1,1) + denorm_lr_predicted_tr
   denorm_lr_predicted_Temp_val = orig_temp_val.reshape(-1,1) + denorm_lr_predicted_val

   del norm_outputs_Temp_tr
   del norm_outputs_Temp_val
   gc.collect()

   np.save('./True_T', denorm_outputs_Temp_tr)
   np.save('./Predicted_T', denorm_lr_predicted_Temp_tr)

   am.get_stats(model_name, 
                name1='Training', truth1=denorm_outputs_Temp_tr, exp1=denorm_lr_predicted_Temp_tr, pers1=predict_persistance_Temp_tr,
                name2='Validation', truth2=denorm_outputs_Temp_val, exp2=denorm_lr_predicted_Temp_val, pers2=predict_persistance_Temp_val,
                name='TrainVal_Temp')

   print('plot results')
   top    = max(max(denorm_outputs_Temp_tr), max(denorm_lr_predicted_Temp_tr), max(denorm_outputs_Temp_val), max(denorm_lr_predicted_Temp_val))
   top    = top + 0.1*abs(top)
   bottom = min(min(denorm_outputs_Temp_tr), min(denorm_lr_predicted_Temp_tr), min(denorm_outputs_Temp_val), min(denorm_lr_predicted_Temp_val))
   bottom = bottom - 0.1*abs(top)
   am.plot_scatter(model_name, denorm_outputs_Temp_tr, denorm_lr_predicted_Temp_tr, name='Temp_train', top=top, bottom=bottom, text='(a)')
   am.plot_scatter(model_name, denorm_outputs_Temp_val, denorm_lr_predicted_Temp_val, name='Temp_val', top=top, bottom=bottom, text='(b)')

   # Get stats and plot anomaly correlation coefficients

   print('clim')
   print(clim_temp_tr[:5])
   print('outputs_Temp')
   print(denorm_outputs_Temp_tr[:5])
   print('predicted_Temp')
   print(denorm_lr_predicted_Temp_tr[:5])

   am.get_stats(model_name, 
                name1='Training', truth1=denorm_outputs_Temp_tr-clim_temp_tr, exp1=denorm_lr_predicted_Temp_tr-clim_temp_tr, 
                pers1=predict_persistance_Temp_tr-clim_temp_tr,
                name2='Validation', truth2=denorm_outputs_Temp_val-clim_temp_val, exp2=denorm_lr_predicted_Temp_val-clim_temp_val,
                pers2=predict_persistance_Temp_val-clim_temp_val,
                name='TrainVal_AnomCCTemp')

   print('plot results')
   top    = max(max(denorm_outputs_Temp_tr-clim_temp_tr), max(denorm_lr_predicted_Temp_tr-clim_temp_tr),
                max(denorm_outputs_Temp_val-clim_temp_val), max(denorm_lr_predicted_Temp_val-clim_temp_val))
   top    = top + 0.1*abs(top)
   bottom = min(min(denorm_outputs_Temp_tr-clim_temp_tr), min(denorm_lr_predicted_Temp_tr-clim_temp_tr), 
                min(denorm_outputs_Temp_val-clim_temp_val), min(denorm_lr_predicted_Temp_val-clim_temp_val))
   bottom = bottom - 0.1*abs(top)
   am.plot_scatter(model_name, denorm_outputs_Temp_tr-clim_temp_tr, denorm_lr_predicted_Temp_tr-clim_temp_tr, 
                   name='AnomCC_train', top=top, bottom=bottom, text='(a)')
   am.plot_scatter(model_name, denorm_outputs_Temp_val-clim_temp_val, denorm_lr_predicted_Temp_val-clim_temp_val,
                   name='AnomCC_val', top=top, bottom=bottom, text='(b)')

del norm_inputs_tr
del norm_inputs_val
del norm_lr_predicted_tr
del norm_lr_predicted_val
gc.collect()

get stats
plot results
plot results
clim
[[16.967073 ]
 [ 4.5871186]
 [ 4.6573954]
 [12.001598 ]
 [17.478577 ]]
outputs_Temp
[[16.980074 ]
 [ 4.5814347]
 [ 4.6574664]
 [12.025378 ]
 [17.472569 ]]
predicted_Temp
[[16.97980705]
 [ 4.58010487]
 [ 4.65754174]
 [12.02586825]
 [17.47296294]]
plot results


12